In [ ]:
import pandas as pd
import re

In [ ]:
def preprocess_informal(text, min_length=6):
    # Xử lý khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng dư thừa
    text = re.sub(r'\s+([?.!,])', r'\1', text)  # Xóa khoảng trắng dư trước dấu câu
    text = re.sub(r'([?.!])\1{2,}', r'\1', text)  # Giảm thiểu dấu câu lặp

    # Thay thế từ viết tắt tạm thời để không bị tách nhầm
    text = re.sub(r'\b(mr|dr|ms|mrs)\.', r'\1<ABBR>', text, flags=re.IGNORECASE)

    # Tách câu dựa trên dấu câu kết thúc
    sentences = re.split(r'(?<=\.|\?|\!)\s', text)

    # Phục hồi từ viết tắt
    sentences = [re.sub(r'<ABBR>', '.', s, flags=re.IGNORECASE) for s in sentences]

    # Chỉ giữ câu dài hơn min_length
    sentences = [s.strip() for s in sentences if len(s.strip()) >= min_length]

    return sentences


In [ ]:
def preprocess_formal(text, min_length=6):

    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s+([?.!,])', r'\1', text)
    text = re.sub(r'([?.!])\1{2,}', r'\1', text)

    # Tách câu theo dấu câu (dấu chấm, chấm hỏi, chấm than)
    # Bảo toàn các định dạng đặc biệt như email, từ viết tắt
    sentences = re.split(r'(?<!\w\.\w)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', text)

    # Chỉ giữ câu dài hơn min_length
    sentences = [s.strip() for s in sentences if len(s.strip()) >= min_length]

    return sentences


In [ ]:
input_file = 'dataset.csv'
df = pd.read_csv(input_file, encoding='utf-8')
df

,Formal,Informal
0,Thân chào bạn,hello cậu
1,Kính gửi cô,gửi cô
2,Trân trọng cảm ơn bạn đã gửi CV ứng tuyển vào ...,cảm ơn bạn đã ứng tuyển vào vị trí Gia Sư nhe
3,Chúng tôi trân trọng mời bạn tham gia phỏng vấ...,bạn tham gia phỏng vấn online cho vị trí nói t...
4,Bạn vui lòng phản hồi mail này để xác nhận tha...,mong bạn trả lời mail để xác nhận tham gia phỏ...
...,...,...
1723,NaN,NaN
1724,NaN,NaN
1725,NaN,NaN
1726,NaN,NaN


In [ ]:
# Thay thế NaN bằng chuỗi rỗng trước khi gọi hàm
df['Informal'] = df['Informal'].fillna("")
df['Formal'] = df['Formal'].fillna("")


In [ ]:
processed_formal = []
processed_informal = []

for sentence in df['Formal']:
    processed_formal.append(preprocess_formal(sentence))

for sentence in df['Informal']:
    processed_informal.append(preprocess_informal(sentence))

In [ ]:
flat_formal = [item for sublist in processed_formal for item in sublist] if any(isinstance(i, list) for i in processed_formal) else processed_formal
flat_informal = [item for sublist in processed_informal for item in sublist] if any(isinstance(i, list) for i in processed_informal) else processed_informal

df_output = pd.DataFrame({'processed_formal': flat_formal, 'processed_informal': flat_informal})

In [ ]:
# Lưu vào file CSV
output_file = 'processed_file.csv'
df_output.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Dữ liệu đã được lưu vào {output_file}")

Dữ liệu đã được lưu vào processed_file.csv
